# Train baseline model for nodule classification

## Setup environment

### [Optional]: Install dependencies

In [ ]:
!pip install "monai[nibabel,skimage,pillow,tqdm]" pytorch_lightning wandb

### [Optional]: Download data

In [ ]:
!mkdir -p ../data/full/processed/images ../data/full/processed/masks ../data/full/cache

In [ ]:
!gsutil cp gs://lung-cancer-detection/lidc-idri/processed/images.zip ../data/full/processed

In [ ]:
!gsutil cp gs://lung-cancer-detection/lidc-idri/processed/masks.zip ../data/full/processed

In [ ]:
!gsutil cp gs://lung-cancer-detection/lidc-idri/processed/meta.zip ../data/full/processed

In [ ]:
!gsutil cp gs://lung-cancer-detection/lidc-idri/splits.zip ../data/full/

In [ ]:
!unzip ../data/full/processed/images.zip -d ../data/full/processed

In [ ]:
!unzip ../data/full/processed/masks.zip -d ../data/full/processed

In [ ]:
!unzip ../data/full/processed/meta.zip -d ../data/full/processed

In [ ]:
!unzip ../data/full/splits.zip -d ../data/full

### [Optional]: Enable module import

In [ ]:
!ln -s ./../lung_cancer_detection

## Load modules and configuration

### Import modules

In [1]:
from pathlib import Path

import numpy as np
import wandb
import torch
from monai.networks.nets import BasicUNet
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger

from lung_cancer_detection.data.scan import SegmentationDataModule
from lung_cancer_detection.models.classification import NoduleSegmentationModel
from lung_cancer_detection.utils import load_config, load_json, preview_dataset

### Load configuration file

In [2]:
cp = Path("../configs/cloud.yaml").absolute()
cp.exists()

True

In [3]:
config = load_config(cp)
config

{'random_seed': 47,
 'wandb': {'offline': False, 'project': 'lung-cancer-detection'},
 'artifacts': {'data': {'name': 'lidc-idri-raw',
   'version': 'v1',
   'type': 'dataset',
   'description': 'Zipped dataset of all chest CT scans, masks and nodule volumes in npy format, including scan and nodule metadata. Updated with new volume size for nodules.'},
  'train': {'name': 'lidc-train',
   'version': 'v1',
   'type': 'dataset',
   'description': 'List of patient IDs included in the training set'},
  'valid': {'name': 'lidc-valid',
   'version': 'v1',
   'type': 'dataset',
   'description': 'List of patient IDs included in the validation set'},
  'class_model': {'name': 'nodule-classification-model',
   'version': 'v0',
   'type': 'model',
   'description': 'Basic DenseNet for classifying lung nodules regarding their malignancy'},
  'seg_model': {'name': 'nodule-segmentation-model',
   'version': 'v1',
   'type': 'model',
   'description': 'Basic UNet for segmenting lung nodules in chest

## Explore datasets

In [4]:
splits = [
    load_json(Path(config["data"]["split_dir"])/"train.json"), 
    load_json(Path(config["data"]["split_dir"])/"valid.json")
]

In [5]:
dm = SegmentationDataModule(
    data_dir=Path(config["data"]["data_dir"]),
    cache_dir=(Path()/"../data/cache/").absolute(),
    splits=splits,
    batch_size=config["data"]["batch_size"]
)

In [6]:
dm.setup()

In [7]:
print(f"Number of training examples: {len(dm.train_ds)}")
print(f"Number of validation examples: {len(dm.val_ds)}")

Number of training examples: 2155
Number of validation examples: 470


## Validation check: overfit one batch

In [10]:
dm = SegmentationDataModule(
    data_dir=Path(config["data"]["data_dir"]),
    cache_dir=(Path()/"../data/cache/").absolute(),
    splits=splits,
    batch_size=config["data"]["batch_size"]
)

In [11]:
net = BasicUNet(
    dimensions=config["seg_model"]["dimensions"],
    in_channels=config["seg_model"]["in_channels"],
    out_channels=config["seg_model"]["out_channels"],
    features=config["seg_model"]["features"],
)

In [12]:
model = NoduleSegmentationModel(net, lr=config["seg_model"]["lr"])

In [13]:
trainer = Trainer(gpus=1, overfit_batches=1, max_epochs=20)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [14]:
trainer.fit(model, datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | DenseNet         | 11.2 M
1 | loss      | CrossEntropyLoss | 0     
2 | train_acc | Accuracy         | 0     
3 | val_acc   | Accuracy         | 0     
-----------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.979    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:342: UserWarning: You requested to overfit but enabled val/test dataloader shuffling. We are turning it off for you.
  'You requested to overfit but enabled val/test dataloader shuffling.'
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:240: UserWarning: You requested to overfit but enabled training dataloader shuffling. We are turning it off for you.
  'You requested to overfit but enabled training dataloader shuffling.'


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

## Run experiment

### Configure experiment

In [88]:
config = load_config(cp)

In [10]:
dm = SegmentationDataModule(
    data_dir=Path(config["data"]["data_dir"]),
    cache_dir=(Path()/"../data/cache/").absolute(),
    splits=splits,
    batch_size=config["data"]["batch_size"]
)

In [11]:
net = BasicUNet(
    dimensions=config["seg_model"]["dimensions"],
    in_channels=config["seg_model"]["in_channels"],
    out_channels=config["seg_model"]["out_channels"],
    features=config["seg_model"]["features"],
)

In [12]:
model = NoduleSegmentationModel(net, lr=config["seg_model"]["lr"])

In [91]:
wandb.login()

True

In [92]:
logger = WandbLogger(project=config["wandb"]["project"], job_type="training")

In [93]:
es = EarlyStopping(monitor="val_loss", verbose=True)
mc = ModelCheckpoint(monitor="val_loss", filename="{epoch}-{step}-{val_loss:.4f}", verbose=True, save_top_k=1)
callbacks = [es, mc]

In [94]:
trainer = Trainer(
    logger=logger,
    callbacks=callbacks,
    **config["experiment"]
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [95]:
trainer.logger.experiment.use_artifact(config["artifacts"]["train"]["name"] + ":" + config["artifacts"]["train"]["version"])
trainer.logger.experiment.use_artifact(config["artifacts"]["valid"]["name"] + ":" + config["artifacts"]["valid"]["version"])

<Artifact QXJ0aWZhY3Q6NDIzMjE4OA==>

### Train model

In [96]:
trainer.fit(model, datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | DenseNet         | 11.2 M
1 | loss      | CrossEntropyLoss | 0     
2 | train_acc | Accuracy         | 0     
3 | val_acc   | Accuracy         | 0     
-----------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.979    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Metric val_loss improved. New best score: 0.646
Epoch 0, global step 67: val_loss reached 0.64597 (best 0.64597), saving model to "/home/jupyter/lung-cancer-detection/nbs/wandb/run-20210712_085128-iru4609w/files/lung-cancer-detection/iru4609w/checkpoints/epoch=0-step=67-val_loss=0.6460-val_acc=0.6170.ckpt" as top 1


...

Epoch 5, global step 407: val_loss was not in top 1


Validating: 0it [00:00, ?it/s]

Epoch 6, global step 475: val_loss was not in top 1


Validating: 0it [00:00, ?it/s]

Monitored metric val_loss did not improve in the last 3 records. Best score: 0.536. Signaling Trainer to stop.
Epoch 7, global step 543: val_loss was not in top 1


### Finish experiment

In [97]:
model_artifact = wandb.Artifact(
    config["artifacts"]["seg_model"]["name"],
    type=config["artifacts"]["seg_model"]["type"],
    description=config["artifacts"]["seg_model"]["description"],
)
model_artifact.add_file(mc.best_model_path)
trainer.logger.experiment.log_artifact(model_artifact)

In [98]:
wandb.finish()

train_loss,0.25847
train_acc,0.96875
epoch,7
trainer/global_step,543
_runtime,144
_timestamp,1626080032
_step,17
val_loss,0.54732
val_acc,0.78298


train_loss,█▇▆▆▇▅▂▃▃▁
train_acc,▁▄▃▃▁▃▆▆▅█
epoch,▁▁▂▂▃▃▃▄▄▅▅▆▆▆▇▇██
trainer/global_step,▁▁▂▂▂▃▃▄▄▅▅▅▆▆▇▇▇█
_runtime,▁▁▂▂▃▃▃▄▄▅▅▅▆▆▇▇▇█
_timestamp,▁▁▂▂▃▃▃▄▄▅▅▅▆▆▇▇▇█
_step,▁▁▂▂▃▃▃▄▄▅▅▆▆▆▇▇██
val_loss,█▃▂▁▁▂▁▂
val_acc,▁▇██████
